In [ ]:
import os
import csv
import xml.etree.ElementTree as ET

def parse_xml_to_connections(input_folder, output_csv):
    csv_data = []
    header = ["Linia", "Typ", "Wariant", "Przystanek startowy ID", "Przystanek startowy nazwa", 
              "Przystanek końcowy ID", "Przystanek końcowy nazwa", "Czas przejazdu"]

    for root_dir, sub_dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith(".xml"):
                file_path = os.path.join(root_dir, file)
                try:
                    tree = ET.parse(file_path)
                    root = tree.getroot()

                    for linia in root.findall('linia'):

                        linia_nazwa = linia.get('nazwa')
                        linia_typ = linia.get('typ')

                        for wariant in linia.findall('wariant'):
                            wariant_nazwa = wariant.get('nazwa')
                            print(f"Przetwarzam wariant: {wariant_nazwa}")
                            
                            przystanki = wariant.findall('.//czasy/przystanek')
                            print(f"Liczba przystanków w wariancie {wariant_nazwa}: {len(przystanki)}")

                            for i in range(len(przystanki) - 1):
                                start_przystanek = przystanki[i]
                                koncowy_przystanek = przystanki[i + 1]

                                start_id = start_przystanek.get('id')
                                start_nazwa = start_przystanek.get('nazwa')
                                try:
                                    start_czas = int(start_przystanek.get('czas', 0))
                                except ValueError:
                                    start_czas = 0  

                                koniec_id = koncowy_przystanek.get('id')
                                koniec_nazwa = koncowy_przystanek.get('nazwa')
                                try:
                                    koniec_czas = int(koncowy_przystanek.get('czas', 0))
                                except ValueError:
                                    koniec_czas = 0 

                                czas_przejazdu = koniec_czas - start_czas

                                csv_data.append([
                                    linia_nazwa, linia_typ, wariant_nazwa, 
                                    start_id, start_nazwa, 
                                    koniec_id, koniec_nazwa, 
                                    czas_przejazdu
                                ])

                except Exception as e:
                    print(f"Błąd podczas przetwarzania pliku {file_path}: {e}")

    with open(output_csv, mode='w', encoding='utf-8', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        writer.writerows(csv_data)
    print(f"Zakończono przetwarzanie. Zapisano {len(csv_data)} połączeń do pliku CSV.")

input_folder = "XML-rozkladyjazdy" 
output_csv = "graf_polaczen.csv" 
parse_xml_to_connections(input_folder, output_csv)


Przetwarzam wariant: DWORZEC GŁÓWNY - ZOO
Liczba przystanków w wariancie DWORZEC GŁÓWNY - ZOO: 152
Przetwarzam wariant: ZOO - DWORZEC GŁÓWNY
Liczba przystanków w wariancie ZOO - DWORZEC GŁÓWNY: 65
Przetwarzam wariant: DWORZEC GŁÓWNY - Zajezdnia BOREK
Liczba przystanków w wariancie DWORZEC GŁÓWNY - Zajezdnia BOREK: 68
Przetwarzam wariant: Zajezdnia BOREK - ZOO
Liczba przystanków w wariancie Zajezdnia BOREK - ZOO: 299
Przetwarzam wariant: BISKUPIN - KRZYKI
Liczba przystanków w wariancie BISKUPIN - KRZYKI: 299
Przetwarzam wariant: KRZYKI - BISKUPIN
Liczba przystanków w wariancie KRZYKI - BISKUPIN: 299
Przetwarzam wariant: BISKUPIN - Zajezdnia OŁBIN
Liczba przystanków w wariancie BISKUPIN - Zajezdnia OŁBIN: 222
Przetwarzam wariant: BISKUPIN - Zajezdnia GAJ
Liczba przystanków w wariancie BISKUPIN - Zajezdnia GAJ: 269
Przetwarzam wariant: KRZYKI - Zajezdnia OŁBIN
Liczba przystanków w wariancie KRZYKI - Zajezdnia OŁBIN: 244
Przetwarzam wariant: Zajezdnia GAJ - KRZYKI
Liczba przystanków w wari